In [ ]:
import plotly.express as px
import pandas as pd

In [21]:
data = pd.read_csv(r"/Users/Jimeno/Google Drive/Fonnachts/02_Presupuesto/data/OUTPUTS/registry_investments.csv")
data['DATE'] = pd.to_datetime(data['DATE'], format='%Y-%m-%d')
data['YEAR'] = data['DATE'].dt.year
data.head()

,Unnamed: 0,DATE,CREDIT,DEBIT,BALANCE,CURRENCY,CAT,TYPE,ACCOUNT,DESC_1,...,BALANCE_SGD,CREDIT_CHF,DEBIT_CHF,BALANCE_CHF,CREDIT_COP,DEBIT_COP,BALANCE_COP,CREDIT_USD,DEBIT_USD,BALANCE_USD
0,0,2021-02-15,850.0,0.0,850.0,CHF,Initial Investment,RETIREMENT,JF-EBP,NaN,...,1264.63,850.0,0.0,850.0,2741935.525,0.0,2.741936e+06,949.620,0.0,949.620
1,1,2021-03-15,850.0,0.0,1700.0,CHF,Initial Investment,RETIREMENT,JF-EBP,NaN,...,2476.05,850.0,0.0,1700.0,2741935.525,0.0,5.483871e+06,930.155,0.0,1860.310
2,2,2021-04-15,850.0,0.0,2550.0,CHF,Initial Investment,RETIREMENT,JF-EBP,NaN,...,3630.18,850.0,0.0,2550.0,2741935.525,0.0,8.225807e+06,899.555,0.0,2698.665
3,3,2021-05-15,850.0,0.0,3400.0,CHF,Initial Investment,RETIREMENT,JF-EBP,NaN,...,4951.76,850.0,0.0,3400.0,2741935.525,0.0,1.096774e+07,929.305,0.0,3717.220
4,4,2021-06-15,850.0,0.0,4250.0,CHF,Initial Investment,RETIREMENT,JF-EBP,NaN,...,6255.15,850.0,0.0,4250.0,2741935.525,0.0,1.370968e+07,945.880,0.0,4729.400


In [22]:
data.CAT.unique()

array(['Initial Investment', 'Interest Earned', 'Closing', 'Losses',
       'Cashout', 'O&M Real Estate', 'Financing'], dtype=object)

In [23]:
data.ACCOUNT.unique()

array(['JF-EBP', 'JF-PUBLICA', 'LF-PUBLICA', 'American Portfolio',
       'Asian Portfolio', 'CDT3744-S43700', 'CDT3843-S1160',
       'CDT2900-S1164', 'CDT8147-S1164', 'CDT215-2772',
       'POST-92-78-5858-6', 'DBS-120-244451-15', 'CDT-212960',
       'CDT-214370', 'CDT-211160', 'MANUL-2450472265', 'MANUL-2450472273',
       'APT501', 'NIDAU', 'LAX'], dtype=object)

In [24]:
year = "2021"
Month = "Jun"
account = "CDT-214370" #""CDT3744-S43700" "CDT-214370
accounts = ['JF-EBP', 'JF-PUBLICA', 'LF-PUBLICA', 'American Portfolio',
       'Asian Portfolio', 'CDT3744-S43700', 'CDT3843-S1160',
       'CDT2900-S1164', 'CDT8147-S1164', 'CDT215-2772',
       'POST-92-78-5858-6', 'DBS-120-244451-15', 'CDT-212960',
       'CDT-214370', 'CDT-211160', 'MANUL-2450472265', 'MANUL-2450472273',
       'APT501', 'NIDAU', 'LAX']
currency = "CHF"
report = pd.to_datetime('2021-08-08 00:00:00')

In [39]:
def age (date, date_closing):
    diffence_date = date_closing - date
    difference_in_fraction_year = diffence_date.days /365
    return difference_in_fraction_year

result = pd.DataFrame()
for i, account in enumerate(accounts):
    print(account)
    data_account = data[data["ACCOUNT"]== account].reset_index(drop=True)
    type_account = data_account["TYPE"][0]
    
    data_account_is_closed = ('Closing' in (data_account["CAT"].values))
    latest_date_available = data_account["DATE"].max()
    date_opening = data_account.loc[0, "DATE"]
    
    if report < date_opening:
        print("This account was not open by the date of reporting, report not possible")
        capital = None
        returns = None
        costs = None
        profits = None
        roi_to_date = None
        roi_per_year = None
    else:
        if data_account_is_closed:
            print("This account has been already closed")
            index_closing = data_account.index[data_account["CAT"]== 'Closing'][0]
            date_closing = data_account.loc[index_closing, "DATE"]
            data_requested_earlier_than_closed = date_closing >= report
            if data_requested_earlier_than_closed:
                date_closing = report
            else:
                print("The account is already closed by the reporting date, reporting for the closing date")
        else:
            print("The account is open")
            if latest_date_available >= report:
                date_closing = report
            else:
                print("The reporting date goes beyond the latest available date, reporting for latest date")
                date_closing = latest_date_available
        print(date_closing)
        
        data_account = data_account[data_account["DATE"] <= date_closing]
        investments = data_account.loc[data_account["CAT"] == "Initial Investment"]
        index_first_investement = investments.index[investments["DATE"] == investments["DATE"].min()][0]
        date_first_investement = investments.loc[index_first_investement, "DATE"]
        number_years_invested = age(date_first_investement, date_closing)
        
        capital = investments["CREDIT_" + currency].sum()
        capital -= data_account.loc[data_account["CAT"] == "Financing"]["DEBIT_" + currency].sum()
        returns = data_account.loc[data_account["CAT"] == "Interest Earned"]["CREDIT_" + currency].sum()
        costs = data_account.loc[data_account["CAT"] == "O&M Real Estate"]["CREDIT_" + currency].sum()
        costs += data_account.loc[data_account["CAT"] == 'Losses']["CREDIT_" + currency].sum()
        profits = returns - costs
        roi_to_date = profits / capital * 100
        roi_per_year = roi_to_date / number_years_invested
        profits_per_year = profits / number_years_invested

        result = result.append(pd.DataFrame({"year": year,
                                            "account": account,
                                             "type":type_account,
                                             "closed": data_account_is_closed,
                                             "last date":date_closing,
                                "capital":capital,
                                "costs":costs,
                                "profits (total to date)":profits,
                                "profits (average per year)":profits_per_year,
                                "return on capital (total to date)":roi_to_date,
                                "return on capital (average per year)":roi_per_year}, index=[i]))

JF-EBP
The account is open
2021-08-08 00:00:00
JF-PUBLICA
This account has been already closed
The account is already closed by the reporting date, reporting for the closing date
2020-10-09 00:00:00
LF-PUBLICA
The account is open
The reporting date goes beyond the latest available date, reporting for latest date
2021-08-05 00:00:00
American Portfolio
The account is open
The reporting date goes beyond the latest available date, reporting for latest date
2021-07-09 00:00:00
Asian Portfolio
The account is open
The reporting date goes beyond the latest available date, reporting for latest date
2021-07-09 00:00:00
CDT3744-S43700
This account has been already closed
The account is already closed by the reporting date, reporting for the closing date
2018-01-15 00:00:00
CDT3843-S1160
This account has been already closed
The account is already closed by the reporting date, reporting for the closing date
2018-01-15 00:00:00
CDT2900-S1164
This account has been already closed
The account is alread

In [41]:
fig = px.bar(result[result["closed"]==False], x='account', y='return on capital (total to date)', color="type", text="last date")
fig.show()

In [42]:
fig = px.bar(result[result["closed"]==False], x='account', y='return on capital (average per year)', color="type", text="last date")
fig.show()

In [44]:
fig = px.bar(result[result["closed"]==False], x='account', y='profits (average per year)', color="type")
fig.show()

In [43]:
fig = px.bar(result[result["closed"]==False], x='account', y='capital', color="type")
fig.show()

In [45]:
fig = px.bar(result[result["closed"]==False], x='account', y='profits (total to date)', color="type")
fig.show()

In [47]:
total_capital = result[result["closed"]==False]["capital"].sum()
total_profits = result[result["closed"]==False]["profits (total to date)"].sum()
total_roi = total_profits/total_capital
total_roi * 100

8.530724522387228